<a href="https://colab.research.google.com/github/coltoncandy/AI-Music-Generation-Research/blob/main/Genetic_Music_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
PROJECT_PATH = "/content/gdrive/Shareddrives/AI Winter 2021 Group Project/Genetic Algorithm 1.0.0"

Mounted at /content/gdrive


In [2]:
from music21 import converter, note, stream, duration

# this should be a list of tuples
# the tuples should be as follows: (r, m, l)
# where r is a boolean being True if it is a rest note
# m is a midi note number from 0-127 (it is ignored if r is true)
# l is the length in 16ths (it should range from 1-16)
notes = []

# Colton fix this please, I'm not sure how to get the right directory for the midi songs from google drive
# dos.mid is just a random one I chose
input = converter.parse("midi_songs/dos.mid").flat.elements

# for each note in the input stream, turn it into a tuple
# ignore anything that is not a note or a rest
for n in input:
    if isinstance(n, note.Note):
        notes.append((False, n.pitch.midi, n.quarterLength * 4))
    if isinstance(n, note.Rest):
        notes.append((True, 0, n.quarterLength * 4))

output = stream.Stream()

for cur_note in notes:
    new_note = None

    if cur_note[0]:
        new_note = note.Rest(duration=duration.Duration(cur_note[2]/4))
    else:
        new_note = note.Note(cur_note[1], duration=duration.Duration(cur_note[2]/4))
    
    output.append(new_note)

output.show('midi')
output.write('midi', fp='test.midi')

music21: Certain music21 functions might need these optional packages: matplotlib, numpy;
                   if you run into errors, install them by following the instructions at
                   http://mit.edu/music21/doc/installing/installAdditional.html


'test.midi'